In [ ]:
import glob

from netCDF4 import Dataset
from scipy.ndimage import rotate, maximum_filter

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation

from scipy.interpolate import RectBivariateSpline

import meshio

%matplotlib inline

In [ ]:
def jacobian(x0, y0, x1, y1, x2, y2):
    """
    jac = jacobian(x0, y0, x1, y1, x2, y2):
    calculates jac = det(M),
    where M is the matrix
    [[x1-x0, y1-y0], [x2-x0, y2-y0]].

    This is twice the area of a triangle with vertices:
    (x0, y0), (x1, y1), (x2, y2)

    Parameters:
    x0, x1, x2 (numpy arrays or floats) - x coords of the 3 points
    y0, y1, y2 (numpy arrays or floats) - y coords of the 3 points

    Returns:
    jac (same type as inputs)
    """
    return (x1-x0)*(y2-y0)-(x2-x0)*(y1-y0)


def get_areas(self):
    ''' Get areas of elements '''
    indices = self.indices

    areas = []
    for inds in indices:
        jac = self.get_jacobian(self.nodes_x, self.nodes_y, inds)
        areas.append( .5*np.abs(jac) )
    return areas

def get_jacobian(self, nodes_x, nodes_y, inds):
    """
    interface to calculate the jacobian for a single element
    self.get_jacobian(nodes_x,nodes_y,inds)

    Parameters:
    -----------
    nodes_x : numpy.ndarray
        x coordinates of mesh nodes
    nodes_y : numpy.ndarray
        y coordinates of mesh nodes
    inds : numpy.ndarray(int)
        - one row of self.indices
        - three elements
           - one for each node of the current triangular element
    """
    xy = []
    for n in inds:
        xy.append(nodes_x[n])
        xy.append(nodes_y[n])
    return jacobian(*xy)

def clean_mesh(tri):
    """
    Remove unused points from a mesh and reindex triangulation

    Parameters:
    x, y (numpy arrays) - coordinates of all points
    t (numpy array) - triangulation indices

    Returns:
    x_new, y_new, t_new - cleaned mesh data
    """
    x, y, t = tri.x, tri.y, tri.triangles
    # Find all unique indices that are used in the triangulation
    used_indices = np.unique(t)

    # Create a mapping from old indices to new indices
    index_map = np.zeros(len(x), dtype=np.int32) - 1
    index_map[used_indices] = np.arange(len(used_indices))

    # Create new coordinate arrays containing only used points
    x_new = x[used_indices]
    y_new = y[used_indices]

    # Reindex triangulation
    t_new = index_map[t]

    #print(f"Removed {len(x) - len(x_new)} unused points from mesh")
    return Triangulation(x_new, y_new, t_new)

def measure(x, y, t):
    area = .5*np.abs(jacobian(x[t][:,0], y[t][:,0], x[t][:,1], y[t][:,1], x[t][:,2], y[t][:,2]))
    dx = np.diff(np.hstack([x[t], x[t][:,0][None].T]))
    dy = np.diff(np.hstack([y[t], y[t][:,0][None].T]))
    edges = np.hypot(dx, dy)
    perim = edges.sum(axis=1)
    ap_ratio = area**0.5/ perim
    return area, edges, perim, ap_ratio


In [ ]:
#ice_conc_sh_ease2-250_cdr-v3p0_197903011200.nc
# load maximum concentration and average
idir = '/Volumes/sim/data/OSISAF_ice_conc_CDR_v3p0/1979/09'
ifiles = sorted(glob.glob(f'{idir}/ice_conc_sh_*nc'))
c = np.dstack([Dataset(ifile)['ice_conc'][0].filled(np.nan) for ifile in ifiles])
c = np.nanmean(c, axis=2)

In [ ]:
ds = Dataset(ifiles[0])
xc = ds['xc'][:]
yc = ds['xc'][:]

In [ ]:
xlim = [100, 250]
ylim = [50, 200]

plt.figure(figsize=(10,10))
plt.imshow(c, interpolation='nearest')
plt.xticks(range(0,400,10), rotation=90)
plt.yticks(range(0,400,10))
plt.xlim(xlim)
plt.ylim(ylim[::-1])
plt.show()

In [ ]:
mask = np.isfinite(c).astype(int)
mask[:ylim[0]] = 0
mask[ylim[1]:] = 0
mask[:, :xlim[0]] = 0
mask[:, xlim[1]:] = 0
mask[180:, :150, ] = 0
mask[170:, :130, ] = 0
mask[160:, :120, ] = 0
mask[150:, :116, ] = 0


plt.figure(figsize=(10,10))
plt.imshow(mask, interpolation='nearest')

plt.xticks(range(0,400,10))
plt.yticks(range(0,400,10))
plt.xlim(xlim)
plt.ylim(ylim[::-1])
plt.show()

In [ ]:
mask2 = maximum_filter(mask, 7)
plt.figure(figsize=(10,10))
plt.imshow(mask2, interpolation='nearest')

plt.xticks(range(0,400,10))
plt.yticks(range(0,400,10))
plt.xlim(xlim)
plt.ylim(ylim[::-1])
plt.show()

In [ ]:
rows, cols = np.where(mask2[::2, ::2])
xgrd, ygrd = np.meshgrid(xc[::2], yc[::-2])
x0 = xgrd[rows, cols]
y0 = ygrd[rows, cols]
plt.figure(figsize=(20,20))
plt.plot(x0, y0, '.')
plt.show()
print(x0.size)

In [ ]:
t0 = Triangulation(x0, y0).triangles
plt.figure(figsize=(20,20))
plt.triplot(x0, y0, t0)
plt.show()


In [ ]:
x0 += np.random.randn(x0.size)
y0 += np.random.randn(y0.size)
t0 = Triangulation(x0, y0).triangles

a, e, p, r = measure(x0, y0, t0)
fig, ax = plt.subplots(1,4, figsize=(12,3))
ax[0].hist(a, 100, np.percentile(a, [1,99]))
ax[1].hist(e.flatten(), 100, np.percentile(e, [1,99]))
ax[2].hist(p, 100, np.percentile(p, [1,99]))
ax[3].hist(r, 100, np.percentile(r, [1,99]))
plt.show()

In [ ]:
a.min(), a.max(), np.median(a), np.std(a), np.percentile(a, 99)

In [ ]:
max_area = np.percentile(a, 99.9)
min_ap_ratio = 0.19
good_elems = (a < max_area) * (r > min_ap_ratio)
t0 = t0[good_elems]

a, e, p, r = measure(x0, y0, t0)
fig, ax = plt.subplots(1,4, figsize=(12,3))
ax[0].hist(a, 100, np.percentile(a, [0.1,99.9]))
ax[1].hist(e.flatten(), 100, np.percentile(e, [0.1,99.9]))
ax[2].hist(p, 100, np.percentile(p, [0.1,99.9]))
ax[3].hist(r, 100, np.percentile(r, [0.1,99.9]))
plt.show()


In [ ]:
plt.figure(figsize=(20,20))
plt.tripcolor(x0, y0, t0, a)
plt.show()

In [ ]:
plt.figure(figsize=(7,7))
plt.imshow(mask2, interpolation='nearest', extent=(xc[0], xc[-1], yc[0], yc[-1]), origin='upper')
plt.triplot(x0, y0, t0)
plt.show()

In [ ]:
np.savetxt('mesh0x.txt', x0, header=str(x0.size), comments='')
np.savetxt('mesh0y.txt', y0, header=str(y0.size), comments='')
np.savetxt('mesh0t.txt', t0.flatten()+1, header=str(t0.size), comments='', fmt='%d')

In [ ]:
with open('mesh1x.txt') as f:
    x1 = np.loadtxt(f)[1:]
with open('mesh1y.txt') as f:
    y1 = np.loadtxt(f)[1:]
with open('mesh1t.txt') as f:
    t1 = np.loadtxt(f).astype(int)
    t1 = t1[1:].reshape(-1, 3) - 1

In [ ]:
pts0 = np.column_stack((x0, y0))
mesh = meshio.Mesh(
    pts0,
    [("triangle", t0)],
)

mesh.write("weddel0.mesh")

In [ ]:
!mmg2d -hausd 20 -hmin 20 -hmax 26 -in weddel0.mesh -out weddel1.mesh

In [ ]:
mesh1 = meshio.read("weddel1.mesh")
x1, y1 = mesh1.points.T
t1 = mesh1.cells_dict["triangle"]
good_t1_idx = np.nonzero(np.all((t1 <= x1.size), axis=1))[0]
print(t1.shape, good_t1_idx.size)
t1 = t1[good_t1_idx]

In [ ]:
plt.figure(figsize=(20,20))
plt.triplot(x1, y1, t1)
#plt.xlim([0,1000])
#plt.ylim([3000,4000])
plt.plot(x1, y1, '.')
plt.show()

In [ ]:
a, e, p, r = measure(x1, y1, t1)
fig, ax = plt.subplots(1,4, figsize=(12,3))
ax[0].hist(a, 100, range=np.percentile(a, [0.1,99.9]))
ax[1].hist(e.flatten(), 100, range=np.percentile(e, [0.1,99.9]))
ax[2].hist(p, 100, range=np.percentile(p, [0.1,99.9]))
ax[3].hist(r, 100, range=np.percentile(r, [0.1,99.9]))
plt.show()


In [ ]:
#mask[np.isnan(c)] = -1
plt.imshow(mask)

In [ ]:
rbs = RectBivariateSpline(xc, yc, mask[::-1], kx=1, ky=1)
m1 = rbs(y1, x1, grid=False)

In [ ]:
plt.figure(figsize=(5,5))
plt.triplot(x1, y1, t1)
scat = plt.scatter(x1,y1,10,m1)
plt.show()

In [ ]:
minm = 0.5
plt.figure(figsize=(15,15))
plt.triplot(x1, y1, t1)
scat = plt.scatter(x1,y1,10,m1>minm)
plt.xlim([-3000,-1000])
plt.ylim([0,2500])
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
minm = 0.5
plt.figure(figsize=(15,15))
plt.triplot(x1, y1, t1)
scat = plt.scatter(x1,y1,10,m1>minm)
plt.xlim([-3000,-1000])
plt.ylim([2500, 4300])
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
minm = 0.5
plt.figure(figsize=(15,15))
plt.triplot(x1, y1, t1)
scat = plt.scatter(x1,y1,10,m1>minm)
plt.xlim([-1000, 1000])
plt.ylim([0,2500])
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
minm = 0.5
plt.figure(figsize=(15,15))
plt.triplot(x1, y1, t1)
scat = plt.scatter(x1,y1,10,m1>minm)
plt.xlim([-1000, 1000])
plt.ylim([2500, 4300])
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
np.savez('mesh_weddel_ease_25km_mmg_20_26.npz', x=x1, y=y1, t=t1, mask=mask, xc=xc, yc=yc, landmask=np.isnan(c))